# Generalised linear models

This section will focus on poisson regression, which is a generalised linear model (GLM) for count data. 

<!-- We will first introduce the poisson distribution and then show how to fit a poisson regression model to data. We will also discuss the interpretation of the fitted model. -->

## Count data

In previous chapters, we learnt about linear regression for predicting quantitative (continuous) variables and logistic regression for predicting qualitative (categorical) variables. In this section, we will learn about regression for predicting count data. Count data is a special type of discrete data, where the values are non-negative integers. Examples of count data include the number of cars passing a toll booth, the number of people in a room, the number of times a word appears in a document, the number of times a user clicks on a link, and the number of times a user purchases a product.

To study the problem of predicting count data, we will use the [Bikeshare dataset](https://github.com/pykale/transparentML/blob/main/data/Bikeshare.csv). The response is `bikers`, the number of hourly users of a bike sharing program in Washington, DC. Thus, this response variable is neither qualitative nor quantitative. Instead, it takes on non-negative integer values, or counts. We will consider predicting `bikers` using the features `mnth` (month of the year), `hr` (hour of the day, from 0 to 23), `workingday` (an indicator variable that equals 1 if it is neither a weekend nor a holiday), `temp` (the normalised temperature, in Celsius), and `weathersit` (a qualitative variable that takes on one of four possible values: clear; misty or cloudy; light rain or light snow; or heavy rain or heavy snow).


Get ready by importing the APIs needed from respective libraries.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.formula.api import ols, poisson

%matplotlib inline

Load the data, indicate the type of the `mnth` and `hr` features, and display the first few rows.

In [ ]:
data_url = "https://github.com/pykale/transparentML/raw/main/data/Bikeshare.csv"

data_df = pd.read_csv(data_url, header=0, index_col=0)
data_df["mnth"] = data_df["mnth"].astype("category")
data_df["hr"] = data_df["hr"].astype("category")
data_df.head(3)

## Linear regression for predicting the number of bikers

As in the case of motivating logistic regression, let us predict `bikers` using linear regression first using the `ols` function from the `statsmodels` library. We will use the `mnth`, `hr`, `workingday`, `temp`, and `weathersit` features as predictors.

In [ ]:
est = ols("bikers ~ mnth + hr + temp + workingday + weathersit", data_df).fit()
est.summary()

From the above, we can see that a progression of weather from clear to cloudy results in 12.89 fewer bikers per hour on average. If the weather progresses further (from cloudy) to light rain or snow, then this further results in $-12.89 - (-66.49) = 53.60$ fewer bikers per hour.

Let us study the coefficients of the `mnth` and `hr` features to understand the effect of these features on the response. 

In [ ]:
# months = ["Jan", "Feb", "March", "April", "May", "June", "July", "Aug", "Sept", "Oct", "Nov", "Dec"]
months = [
    "Jan",
    "Feb",
    "March",
    "May",
    "June",
    "July",
    "Aug",
    "Sept",
    "Oct",
    "Nov",
    "Dec",
]
coef_mnth = [est.params["mnth[T.%s]" % _m] for _m in months]
coef_hr = [est.params["hr[T.%d]" % _h] for _h in range(1, 24)]

Visualise the coefficients of the `mnth` and `hr` features using plots.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.plot(months, coef_mnth, "bo-")
ax1.grid()
ax1.set_xlabel("Month")
ax2.plot(np.arange(1, 24), coef_hr, "bo-")
ax2.grid()
ax2.set_xlabel("Hour")

for ax in fig.axes:
    ax.set_ylabel("Coefficient")

plt.show()

From the above figures, we can see that bike usage is highest in the spring and fall, and lowest during the winter months. Furthermore, bike usage is greatest around rush hour (9 AM and 6 PM), and lowest overnight. Thus, at first glance, fitting a linear regression model to the `Bikeshare` dataset seems to provide reasonable and intuitive results.

Let us examine the distribution of the estimated response values. We will use the `predict` function from the `statsmodels` library to obtain the predicted values.

In [ ]:
fig = plt.figure(figsize=(12, 7))
y_pred = est.predict(data_df.loc[:, ["mnth", "hr", "temp", "workingday", "weathersit"]])
sns.histplot(y_pred, kde=True)
plt.show()

By careful inspection, we can identify several issues from the figure above, e.g. 9.6% of the fitted values are negative! Thus, the linear regression model predicts a negative number of users during 9.6% of the hours in the dataset, which does not make sense. Furthermore, the fitted values are not integers, which is also not desirable. 

This casts doubt on the suitability of linear regression for performing meaningful predictions on the `Bikeshare` data. It also raises concerns about the accuracy of the coefficient estimates, confidence intervals, and other outputs of the regression model.

Let us further examine the distribution of the response variable `bikers` against the `hr` feature using the original scale and the log scale.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(25, 7))

sns.stripplot(data=data_df, x="hr", y="bikers", ax=ax1, alpha=0.1, color=".2")
sns.boxplot(
    data=data_df,
    x="hr",
    y="bikers",
    ax=ax1,
    width=0.9,
    palette="vlag",
    meanline=True,
    showmeans=True,
)
ax1.set_ylabel("Number of Bikers")

sns.stripplot(data=data_df, x="hr", y="bikers", ax=ax2, alpha=0.1, color=".2")
sns.boxplot(
    data=data_df,
    x="hr",
    y="bikers",
    ax=ax2,
    width=0.9,
    palette="vlag",
    meanline=True,
    showmeans=True,
)
ax2.set_yscale("log")
ax2.set_ylabel("log(Number of Bikers)")
for ax in fig.axes:
    ax.set_xlabel("Hour")
plt.show()

This figure corresponds to Figure 4.14 in the textbook. Intuitively, when the expected number of bikers is small, its variance should be small as well. For example, at 2 AM during a heavy December snow storm, we expect that extremely few people will use a bike, and moreover that there should be little variance associated with the number of users during those conditions. This is borne out in the data: between 1 AM and 4 AM, in December, January, and February, when it is raining, there are 5.05 users, on average, with a standard deviation of 3.73. By contrast, between 7 AM and 10 AM, in April, May, and June, when skies are clear, there are 243.59 users, on average, with a standard deviation of 131.7. The mean-variance relationship is displayed in the left-hand panel of the figure above. This is a major violation of the assumptions of a linear model, which state that $y = \sum_{d=1}^{D}x_d\beta_d + \epsilon$, where $\epsilon$ is a mean-zero error term with variance $\sigma^2$ that is constant, and not a function of the features (covariates). Therefore, the heteroscedasticity of the data calls into question the suitability of a linear regression model.

Finally, the response `bikers` is integer-valued. But under a linear model, $y = \beta_0 + \sum_{d=1}^{D}x_d\beta_d + \epsilon$, where $\epsilon$ is a continuous-valued error term. This means that in a linear model, the response $y$ is necessarily continuous-valued (quantitative). Thus, the integer nature of the response `bikers` suggests that a linear regression model is not entirely satisfactory for this dataset.

Some of the problems that arise when fitting a linear regression model to the Bikeshare data can be overcome by transforming the response; for instance, we can fit the model

$$
\log(y) = \beta_0 + \sum_{d=1}^{D}x_d\beta_d + \epsilon.
$$

Transforming the response _avoids the possibility of negative predictions_, and it _overcomes much of the heteroscedasticity in the untransformed data_, as is shown in the right-hand panel of the figure above. 

However, it is not quite a satisfactory solution, since predictions and inference are made in terms of the log of the response, rather than the response. This leads to challenges in _interpretation_, e.g. a one-unit increase in $x_d$ is associated with an increase in the mean of the log of $y$ by an amount $\beta_d$. Furthermore, a log transformation of the response cannot be applied in settings where the response can take on a value of 0. Thus, while fitting a linear model to a transformation of the response may be an adequate approach for some count-valued datasets, it often leaves something to be desired. This motivates the development of a Poisson regression model that provides a much more natural and elegant approach for this task.

## Poisson regression for predicting the number of bikers


To overcome the inadequacies of linear regression for analysing the `Bikeshare` dataset, we will study a GLM approach called Poisson regression. 

### The Poisson distribution

Let us first introduce the Poisson distribution. Suppose that a random variable $y$ takes on nonnegative integer values, i.e. $y \in \{0, 1, 2, \dots\}$. If $y$ follows the Poisson distribution, then

```{math}
:label: eq:poisson-distribution
\mathbb{P}(y=t) = \frac{\lambda^t e^{-\lambda}}{t!}, \quad t = 0, 1, 2, \dots .
```

<!-- The Poisson distribution is a discrete distribution, and it is characterised by the fact that the probability of observing a particular value of $y$ does not depend on the values of the other random variables. In other words, the probability of observing $y$ is independent of the values of all other random variables. This is in contrast to the binomial distribution, which is also a discrete distribution, but which depends on the values of two random variables: the number of trials, and the probability of success on each trial. The Poisson distribution is also in contrast to the normal distribution, which is a continuous distribution, and which depends on the values of all other random variables. -->

Here, $\lambda>0$ is a positive real number, called the _rate parameter_, which is the expected value of $y$, i.e. $\mathbb{E}(y)$. It turns out that $\lambda$ also equals the variance of $y$, i.e. $λ = \mathbb{E}(y) = \text{Var}(y)$. This means that if $y$ follows the Poisson distribution, then the larger the mean of $y$ , the larger its variance. The notation $ t! $ in Equation {eq}`eq:poisson-distribution` denotes the factorial of $t$, i.e. $ t! = t \times (t − 1) \times (t − 2) \times \cdots \times 3 \times 2 \times 1$. The Poisson distribution is typically used to model counts, a natural choice since counts, like the Poisson distribution, take on non-negative integer values. 

```{admonition} How to interpret the Poisson distribution?
:class: tip
To see how we might use the Poisson distribution in practice, let $y$ denote the number of users of the bike sharing programme during a particular hour of the day, under a particular set of weather conditions, and during a particular month of the year. We can model $y$ as a Poisson distribution with mean $\mathbb{E}(y) = \lambda = 5$. This means that the probability of no users during this particular hour is $\mathbb{P}(y = 0) = \frac{e^{-5}5^0}{0!} = e^{-5} = 0.0067 $, where 0! = 1 by convention. The probability that there is exactly one user is $ \mathbb{P}(y = 1) = \frac{e^{-5}5^1}{1!} = 0.034 $, the probability of two users is $ \mathbb{P}(y = 2) = \frac{e^{-5}5^2}{2!} = 0.084 $ and so on. 
```

### The Poisson regression model

In reality, we expect the mean number of users of the bike sharing program, $ \lambda = \mathbb{E}(y) $, to vary as a function of the hour of the day, the month of the year, the weather conditions, and so forth. So rather than modelling the number of bikers, $y$ , as a Poisson distribution with a _fixed_ mean value like $\lambda = 5 $, we would like to allow the mean to vary as a function of the features (covariates). In particular, we consider the following model for the mean $ \lambda = \mathbb{E}(y) $, which we now write as $ \lambda(x_1, \cdots, x_D) $ to emphasise that it is a function of the features $ x_1, \cdots, x_D $: 

$$
\log(\lambda(x_1, \cdots, x_D)) = \beta_0 + \sum_{d=1}^{D}x_d\beta_d.
$$

or equivalently,

```{math}
:label: eq:poisson-regression
\lambda(x_1, \cdots, x_D) = e^{\beta_0 + \sum_{d=1}^{D}x_d\beta_d}.
```

Here, $ \beta_0, \beta_1, \cdots, \beta_D $ are parameters to be estimated. Together, the above equations define the Poisson regression model.

To estimate the coefficients (parameters) $ \beta_0, \beta_1, \cdots, \beta_D $, we use the same maximum likelihood approach that we adopted for [logistic regression](https://pykale.github.io/transparentML/03-logistic-reg/logistic-regression.html#estimating-the-coefficients-to-make-predictions). The likelihood function for the Poisson regression model is

$$
\ell(\beta_0, \beta_1, \cdots, \beta_D) = \prod_{n=1}^{N} \frac{\lambda(\mathbf{x}_n)^{y_n} e^{-\lambda(\mathbf{x}_n)}}{y_n!},
$$

where $\lambda(\mathbf{x}_n) = e^{\beta_0 + \sum_{d=1}^{D}x_{nd}\beta_d}$, and $y_n$ is the response for the $n\text{th}$ sample (observation). We obtain the coefficient estimates as those maximise the likelihood $\ell(\beta_0, \beta_1, \cdots, \beta_D)$, i.e. those make the observed data as likely as possible.

We now fit a Poisson regression model to the `Bikeshare` dataset using the `statsmodels` package.

In [ ]:
est = poisson("bikers ~ mnth + hr + temp + workingday + weathersit", data_df).fit()
est.summary()

Let us do the same plots as before to visualise the relationship between the number of bikers and the features `mnth` and `hr`.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.plot(months, coef_mnth, "bo-")
ax1.grid()
ax1.set_xlabel("Month")
ax2.plot(np.arange(1, 24), coef_hr, "bo-")
ax2.grid()
ax2.set_xlabel("Hour")

for ax in fig.axes:
    ax.set_ylabel("Coefficient")

plt.show()

We again see that bike usage is highest in the spring and fall and during rush hour, and lowest during the winter and in the early morning hours. In general, bike usage increases as the temperature increases, and decreases as the weather worsens. Interestingly, the coefficient associated with `workingday` is statistically significant under the Poisson regression model, but not under the linear regression model.


### Key distinctions between Poisson and linear regression

Some important distinctions between the Poisson regression model and the linear regression model are as follows:

- _Interpretation_: To interpret the coefficients in the Poisson regression model, we must pay close attention to Equation {eq}`eq:poisson-regression`, which states that an increase in $x_d$ by one unit is associated with a change in $\mathbb{E}(y) = \lambda$ by a factor of $\textrm{exp}(\beta_d)$. For example, a change in weather from clear to cloudy skies is associated with a change in mean bike usage by a factor of $\textrm{exp}(−0.08) = 0.923$, i.e. on average, only 92.3% as many people will use bikes when it is cloudy relative to when it is clear. If the weather worsens further and it begins to rain, then the mean bike usage will further change by a factor of $ \textrm{exp} (−0.5) = 0.607$, i.e. on average only 60.7% as many people will use bikes when it is rainy relative to when it is cloudy.
- _Mean-variance relationship_: As mentioned earlier, under the Poisson model,  $ \lambda = \mathbb{E}(y) = \text{Var}(y) $. Thus, by modelling bike usage with a Poisson regression, we implicitly assume that mean bike usage in a given hour equals the variance of bike usage during that hour. By contrast, under a linear regression model, the variance of bike usage always takes on a constant value. From the above figure (the one corresponding to Figure 4.14 in the textbook), in the `Bikeshare` data, when biking conditions are favourable, both the mean and the variance in bike usage are much higher than when conditions are unfavourable. Thus, the Poisson regression model is able to handle the mean-variance relationship seen in the `Bikeshare` data in a way that the linear regression model is not.
- _Nonnegative fitted values_: There are no negative predictions using the Poisson regression model. This is because the Poisson model itself only allows for nonnegative values, as shown in Equation {eq}`eq:poisson-distribution`. By contrast, when we fit a linear regression model to the `Bikeshare` dataset, almost 10% of the predictions were negative.

## GLM characteristics

Based on the two GLM models we have seen so far, the logistic regression model and the Poisson regression model, we can summarise the common characteristics of GLMs as follows:

1. Each approach uses features (predictors) $ x_1, x_2, \dots, x_D $ to predict a response (output) $ y $. We assume that, conditional on $ x_1, x_2,\dots, x_D $, $ y $ belongs to a certain family of distributions. For linear regression, we typically assume that $y$ follows a Gaussian (normal) distribution. For logistic regression, we assume that $y$ follows a Bernoulli distribution. Finally, for Poisson regression, we assume that $y$ follows a Poisson distribution.
2. Each approach models the mean (expectation) of $y$ as a function of the features (predictors). In linear regression, the mean of $y$ takes the form
    
    $$
    \mathbb{E}(y|x_1, x_2, \dots, x_D) = \beta_0 + \beta_1 x_1 + \cdots + \beta_p x_D.
    $$

    In logistic regression, the mean of $y$ takes the form

    $$
    \mathbb{E}(y|x_1, x_2, \dots, x_D) = \mathbb{P}(y=1|x_1, x_2, \dots, x_D) = \frac{e^{\beta_0 + \beta_1 x_1 + \cdots + \beta_p x_D}}{1 + e^{\beta_0 + \beta_1 x_1 + \cdots + \beta_p x_D}}.
    $$

    In Poisson regression, it takes the form
    
    $$
    \mathbb{E}(y|x_1, x_2, \dots, x_D) = \lambda(x_1, x_2, \dots, x_D) = e^{\beta_0 + \beta_1 x_1 + \cdots + \beta_p x_D}.
    $$

    The above three equations can be expressed using a _link function_, $ \eta $, which applies a transformation to $ \mathbb{E}(y|x_1, x_2, \dots, x_D) $ so that the transformed mean is a linear function of the features (predictors). That is

    ```{math}
    :label: eq:link-function
    \eta(\mathbb{E}(y|x_1, x_2, \dots, x_D)) = \beta_0 + \beta_1 x_1 + \cdots + \beta_p x_D.
    ```

    The link functions for linear, logistic and Poisson regression are $\eta(\mu) = \mu $, $ \eta(\mu) = \log(\mu/(1 − \mu)) $, and $\eta(\mu) = \log(\mu)$, respectively.

The Gaussian, Bernoulli and Poisson distributions are all members of a wider class of distributions, known as the [exponential family](https://en.wikipedia.org/wiki/Exponential_family). Other well known members of this family are the exponential distribution, the Gamma distribution, and the negative binomial distribution. In general, we can perform a regression by modelling the response $y$ as coming from a particular member of the exponential family, and then transforming the mean of the response so that the transformed mean is a linear function of the features (predictors) via the link function above in Equation {eq}`eq:link-function`. Any regression approach that follows this very general recipe is known as a generalized linear model (GLM). Thus, linear regression, logistic regression, and Poisson regression are three examples of GLMs. Other examples not covered here include _Gamma regression_ and _negative binomial regression_.

## Exercises

 **1.** Logistic regression is a generalised linear model (GLM).


       a. True

       b. False
*Compare your answer with the solution below*

 ```{toggle}
    **b. True.**
   ```

**2.** There are no negative predictions using the Poisson regression model.


       a. True

       b. False
*Compare your answer with the solution below*

 ```{toggle}
    **b. True.**
   ```

**3.** For linear regression, we typically assume that $y$ (output) follows a Bernoulli distribution.


       a. True

       b. False
*Compare your answer with the solution below*

 ```{toggle}
    **b. False. Linear regression assumes that the output ($y$) follows a Gaussian(normal) distribution**
   ```